In [24]:
import pandas as pd
from psaw import PushshiftAPI, PushshiftAPIMinimal
import datetime as dt
%pylab inline

api = PushshiftAPI()

In [240]:
from tqdm.notebook import tqdm

In [421]:
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

pio.renderers.default='notebook_connected'

# Get all comments of a subreddit

For the sake of this example, we only take 2018

In [110]:
start_epoch=int(dt.datetime(2018, 1, 1).timestamp())
end_epoch=int(dt.datetime(2019, 1, 1).timestamp())

In [112]:
def all_comments_on_subreddit(subreddit, start= start_epoch, end=end_epoch) : 
    comments = pd.DataFrame(api.search_comments(after=start, before=end, subreddit=subreddit, 
                                                author='![deleted]', filter=['author']))

    del comments['d_']
    del comments['created']
    comments.created_utc = comments.created_utc.apply(dt.datetime.fromtimestamp)
    return comments

In [408]:
def first_comments_in_subreddit(author, subreddit, n=12, end=end_epoch, ) :
    return pd.DataFrame(api.search_comments(author=author, subreddit=subreddit, before=end, 
                        filter=['title'], sort_type='created_utc', sort='asc', limit=n))


In [ ]:
comments = all_comments_on_subreddit('thinkpad')

In [114]:
comments = comments[comments.author != '[deleted]']

comments.to_pickle('thinkpad_2018.pkl')

In [124]:
author_counts = comments.author.value_counts()

In [125]:
# Arbitrary threshold of 1 post/week
author_counts = author_counts[author_counts>52]

# Then remove the top 5% 
author_counts = author_counts[author_counts<author_counts.quantile(.95)]

In [409]:
first_comments = {author:first_comments_in_subreddit(author, 'thinkpad') for author in tqdm(author_counts.index)}

/home/apis/miniconda3/envs/incubator/lib/python3.7/site-packages/psaw/PushshiftAPI.py:153: UserWarning:

Unable to connect to pushshift.io. Retrying after backoff.



In [410]:
prev_activity=dict()
post_activity=dict()

for author in tqdm(first_comments) : 
    first_comment = first_comments[author].created_utc.min() 
    last_comment = first_comments[author].created_utc.max()
    window = dt.timedelta(days=90)
    winseconds = int(window.total_seconds())
    
    agg = next(api.search_comments(author=author, aggs='subreddit', 
                             before=first_comment, after=first_comment-winseconds, limit=0 ))
    
    prev_activity[author] = Counter({rec['key']:rec['doc_count'] for rec in agg['subreddit']})
    
    agg = next(api.search_comments(author=author, aggs='subreddit', 
                             before=last_comment+winseconds, after=last_comment, limit=0 ))
    
    post_activity[author] = Counter({rec['key']:rec['doc_count'] for rec in agg['subreddit']})

In [411]:
cumprev=Counter()
cumpost=Counter()

for author in first_comments :
    prev = prev_activity[author]
    post = post_activity[author]
    cumprev += prev
    cumpost += post

cumprev = pd.Series(cumprev).sort_values(ascending=False)
cumpost = pd.Series(cumpost).sort_values(ascending=False)

In [412]:
del cumpost['thinkpad']
del cumpost['The_Donald']

In [413]:
df = pd.DataFrame({'prev':cumprev, 'post':cumpost})

df.prev /= df.prev.sum()
df.post /= df.post.sum()

df['delta'] = (df.post-df.prev)/df.prev
df = df.sort_values('delta', ascending=False)
df_filtered = df[(df.delta.abs() > df.delta.quantile(0.9))]

In [422]:
go.Figure(go.Bar(x=df_filtered.index, y=df_filtered.delta))

In [423]:
go.Figure(go.Bar(x=df.index, y=df.delta))

In [337]:
?px.histogram

In [245]:
dt.datetime.fromtimestamp(last_comment)

datetime.datetime(2016, 12, 22, 14, 1, 33)

In [247]:
 agg = next(api.search_comments(author=author, aggs='subreddit', 
                             before=first_comment, after=first_comment-winseconds, limit=0 ))

In [230]:
x=dt.timedelta(days=30).total_seconds()

30.0

In [212]:
dt.datetime(2017,1,1).timestamp()

1483246800.0

In [215]:
1483246800-1483246740

60

In [214]:
(dt.datetime(2017,1,1)-dt.timedelta(minutes=1)).timestamp()

1483246740.0

In [169]:
author = 'robodan918'

first_comments[author].timestamp()

1519311197.0

In [160]:
first_comments[author]-dt.timedelta(days=30) 

datetime.datetime(2017, 9, 27, 14, 43, 26)

In [200]:
from collections import Counter
def get_previous_activity (author, date, window=dt.timedelta(days=30)) :
    end = int(date.timestamp())
    begin = int((date-window).timestamp())
    
    
    agg = next(api.search_comments(author='nerpTV', aggs='subreddit', 
                             before=end, after=begin, limit=0 ))
    return Counter({rec['key']:rec['doc_count'] for rec in agg['subreddit']})

In [201]:
for author in  author_counts.sample(3).index :
    print(author, get_previous_activity(author, first_comments[author]))

2018-06-28 09:57:43 2018-07-28 09:57:43
oscareczek Counter({'mac': 46, 'starcitizen': 41, 'thinkpad': 37, 'sysadmin': 30, 'buildapc': 26, 'pcmasterrace': 24, 'Windows10': 20, 'hardware': 17, 'macbook': 15, 'MechanicalKeyboards': 10, 'DataHoarder': 7, 'Dell': 7, 'GlobalOffensive': 7, 'MacOS': 7, 'SuperMegaBaseball': 7, 'WindowsServer': 7, 'GoogleWiFi': 6, 'HomeServer': 6, 'thedivision': 6, 'MacOSBeta': 5, 'PleX': 5, 'apple': 5, 'LogitechG': 4, 'Minecraft': 4, 'pcgaming': 4, 'GameDeals': 3, 'NoMansSkyTheGame': 3, 'macmini': 3, 'nvidia': 3, 'selfhosted': 3, 'CitiesSkylines': 2, 'VAC_Porn': 2, 'buildapcsales': 2, 'Charger': 1, 'ElgatoGaming': 1, 'kindlefire': 1, 'microsoftsoftwareswap': 1, 'patientgamers': 1})
2016-05-06 16:33:45 2016-06-05 16:33:45
hpstr-doofus Counter()
2018-06-26 02:19:54 2018-07-26 02:19:54
harakiriforthemoon Counter({'mac': 41, 'starcitizen': 41, 'thinkpad': 36, 'sysadmin': 33, 'pcmasterrace': 28, 'buildapc': 26, 'hardware': 24, 'Windows10': 22, 'MechanicalKeyboards':